In [1]:
import os
import pickle
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)


In [2]:
# Create results and models folders
RESULTS_DIR = "../results"
MODELS_DIR = "../models"
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# Setup MLflow tracking
mlflow.set_tracking_uri("file:../mlruns")  # logs stored in mlruns folder
mlflow.set_experiment("mlops-assignment-1")


<Experiment: artifact_location='file:///C:/Users/moeda/mlops-assignment-1/src/../mlruns/423857567884080182', creation_time=1758049447579, experiment_id='423857567884080182', last_update_time=1758049447579, lifecycle_stage='active', name='mlops-assignment-1', tags={}>

In [3]:
def train_and_log(model, model_name, X_train, X_test, y_train, y_test):
    """Train a model and log params, metrics, artifacts with MLflow."""
    with mlflow.start_run(run_name=model_name):
        # Train
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average="macro", zero_division=0)
        rec = recall_score(y_test, y_pred, average="macro", zero_division=0)
        f1 = f1_score(y_test, y_pred, average="macro", zero_division=0)

        # Log parameters & metrics
        mlflow.log_params({k: str(v) for k, v in model.get_params().items()})
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        mlflow.log_metric("recall", rec)
        mlflow.log_metric("f1_score", f1)

        # Confusion Matrix plot
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(5, 5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
        plt.title(f"Confusion Matrix - {model_name}")
        plt.xlabel("Predicted")
        plt.ylabel("True")
        cm_path = os.path.join(RESULTS_DIR, f"{model_name}_cm.png")
        plt.savefig(cm_path, bbox_inches="tight")
        plt.close()
        mlflow.log_artifact(cm_path, artifact_path="confusion_matrices")

        # Save model locally & log to MLflow
        local_model_path = os.path.join(MODELS_DIR, f"{model_name}.pkl")
        with open(local_model_path, "wb") as f:
            pickle.dump(model, f)
        mlflow.log_artifact(local_model_path, artifact_path="saved_models")
        mlflow.sklearn.log_model(model, artifact_path=f"sklearn_models/{model_name}")

        print(f"✅ {model_name} logged | acc={acc:.3f}, prec={prec:.3f}, rec={rec:.3f}, f1={f1:.3f}")


In [4]:
# Example dataset (Iris)
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.2, random_state=42, stratify=data.target
)

# Models to compare
models = [
    (LogisticRegression(max_iter=500, solver="liblinear"), "LogisticRegression"),
    (RandomForestClassifier(n_estimators=50, random_state=42, max_depth=2), "RandomForest"),
    (SVC(kernel="rbf", probability=True, gamma=0.7), "SVM"),
]

for model, name in models:
    train_and_log(model, name, X_train, X_test, y_train, y_test)

print("\n🎉 All models logged! Check MLflow UI at http://127.0.0.1:5000")


2025/09/17 10:28:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


MlflowException: Invalid model name ('sklearn_models/LogisticRegression') provided. Model name must be a non-empty string and cannot contain the following characters: ('/', ':', '.', '%', '"', "'")

In [5]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import os

# =====================
# Connect to MLflow Server
# =====================

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("iris-comparison")

# =====================
# Load dataset
# =====================
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

# =====================
# Define models
# =====================
models = {
    "Logistic Regression": LogisticRegression(max_iter=200, solver="lbfgs", multi_class="auto"),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel="linear", probability=True, C=1.0, random_state=42)
}

# =====================
# Directory for plots
# =====================
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

# =====================
# Train & Log
# =====================
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        print(f"Training and logging: {model_name}")
        
        # Train
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average="macro")
        rec = recall_score(y_test, y_pred, average="macro")
        f1 = f1_score(y_test, y_pred, average="macro")

        # Log metrics
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        mlflow.log_metric("recall", rec)
        mlflow.log_metric("f1_score", f1)

        # Log parameters
        if isinstance(model, LogisticRegression):
            mlflow.log_param("C", model.C)
            mlflow.log_param("solver", model.solver)
        elif isinstance(model, RandomForestClassifier):
            mlflow.log_param("n_estimators", model.n_estimators)
            mlflow.log_param("max_depth", model.max_depth)
        elif isinstance(model, SVC):
            mlflow.log_param("kernel", model.kernel)
            mlflow.log_param("C", model.C)

        # Confusion Matrix
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=iris.target_names,
                    yticklabels=iris.target_names)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix - {model_name}")

        plot_path = os.path.join(results_dir, f"{model_name}_confusion_matrix.png")
        plt.savefig(plot_path)
        plt.close()

        # Log artifact
        mlflow.log_artifact(plot_path, artifact_path="plots")

        # Log model
        mlflow.sklearn.log_model(model, artifact_path="model")


2025/09/17 10:36:26 INFO mlflow.tracking.fluent: Experiment with name 'iris-comparison' does not exist. Creating a new experiment.


Training and logging: Logistic Regression


2025/09/17 10:36:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 10:36:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/814975460984374551/runs/659d0cf78fae496aafc8d66fdd264316
🧪 View experiment at: http://localhost:5000/#/experiments/814975460984374551
Training and logging: Random Forest


2025/09/17 10:36:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 10:36:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/814975460984374551/runs/958f2cf9ae404f65a3c0e75aa58930ae
🧪 View experiment at: http://localhost:5000/#/experiments/814975460984374551
Training and logging: SVM


2025/09/17 10:36:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 10:36:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SVM at: http://localhost:5000/#/experiments/814975460984374551/runs/936df62f0fb54a3086369d02bc819767
🧪 View experiment at: http://localhost:5000/#/experiments/814975460984374551


In [6]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import os

# =====================
# 1. Setup MLflow
# =====================

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("iris-comparison")

# =====================
# Load dataset
# =====================
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

# =====================
# Define models
# =====================
models = {
    "Logistic Regression": LogisticRegression(max_iter=200, solver="lbfgs", multi_class="auto"),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel="linear", probability=True, C=1.0, random_state=42)
}

# Directory for plots
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

# =====================
# 2. Train and Log each model
# =====================
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        print(f"Training and logging: {model_name}")

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # ---- Log Metrics ----
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average="macro")
        rec = recall_score(y_test, y_pred, average="macro")
        f1 = f1_score(y_test, y_pred, average="macro")

        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        mlflow.log_metric("recall", rec)
        mlflow.log_metric("f1_score", f1)

        # ---- Log Parameters ----
        if isinstance(model, LogisticRegression):
            mlflow.log_param("C", model.C)
            mlflow.log_param("solver", model.solver)
        elif isinstance(model, RandomForestClassifier):
            mlflow.log_param("n_estimators", model.n_estimators)
            mlflow.log_param("max_depth", model.max_depth)
        elif isinstance(model, SVC):
            mlflow.log_param("kernel", model.kernel)
            mlflow.log_param("C", model.C)

        # ---- Log Artifacts: Confusion Matrix ----
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=iris.target_names,
                    yticklabels=iris.target_names)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix - {model_name}")

        plot_path = os.path.join(results_dir, f"{model_name}_confusion_matrix.png")
        plt.savefig(plot_path)
        plt.close()

        mlflow.log_artifact(plot_path, artifact_path="plots")

        # ---- Log Model ----
        mlflow.sklearn.log_model(model, artifact_path="model")

print("✅ All models trained, logged, and tracked in MLflow.")


Training and logging: Logistic Regression


2025/09/17 10:39:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 10:39:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/814975460984374551/runs/c7e4cd4da5a74c1ca1cda320633ec505
🧪 View experiment at: http://localhost:5000/#/experiments/814975460984374551
Training and logging: Random Forest


2025/09/17 10:39:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 10:40:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/814975460984374551/runs/975f474b501f442d8dda2968fb86a270
🧪 View experiment at: http://localhost:5000/#/experiments/814975460984374551
Training and logging: SVM


2025/09/17 10:40:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 10:40:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SVM at: http://localhost:5000/#/experiments/814975460984374551/runs/ab5e5967dcbd4be5b0e1aab6f1bef908
🧪 View experiment at: http://localhost:5000/#/experiments/814975460984374551
✅ All models trained, logged, and tracked in MLflow.


In [8]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# =====================
# Setup MLflow
# =====================
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("iris-comparison-cv")

# =====================
# Load dataset
# =====================
iris = load_iris()
X, y = iris.data, iris.target

# Stratified K-Fold for fairer evaluation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# =====================
# Define models
# =====================
models = {
    "Logistic Regression": LogisticRegression(max_iter=200, solver="lbfgs", multi_class="auto"),
    "Random Forest": RandomForestClassifier(n_estimators=50, max_depth=3, random_state=42),
    "SVM": SVC(kernel="linear", probability=True, C=0.5, random_state=42)
}

# Directory for plots
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

# =====================
# Train and Log each model with CV
# =====================
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        print(f"🚀 Training and logging with CV: {model_name}")

        # Cross-validation predictions
        y_pred = cross_val_predict(model, X, y, cv=cv)

        # ---- Metrics ----
        acc = accuracy_score(y, y_pred)
        prec = precision_score(y, y_pred, average="macro")
        rec = recall_score(y, y_pred, average="macro")
        f1 = f1_score(y, y_pred, average="macro")

        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        mlflow.log_metric("recall", rec)
        mlflow.log_metric("f1_score", f1)

        # ---- Parameters ----
        if isinstance(model, LogisticRegression):
            mlflow.log_param("C", model.C)
            mlflow.log_param("solver", model.solver)
        elif isinstance(model, RandomForestClassifier):
            mlflow.log_param("n_estimators", model.n_estimators)
            mlflow.log_param("max_depth", model.max_depth)
        elif isinstance(model, SVC):
            mlflow.log_param("kernel", model.kernel)
            mlflow.log_param("C", model.C)

        # ---- Confusion Matrix ----
        cm = confusion_matrix(y, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=iris.target_names,
                    yticklabels=iris.target_names)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix - {model_name} (CV)")

        plot_path = os.path.join(results_dir, f"{model_name}_confusion_matrix.png")
        plt.savefig(plot_path)
        plt.close()

        mlflow.log_artifact(plot_path, artifact_path="plots")

        # ---- Log Model ----
        mlflow.sklearn.log_model(model, artifact_path="model")

print("\n🎯 All models trained with CV, logged, and tracked in MLflow.")


2025/09/17 12:07:27 INFO mlflow.tracking.fluent: Experiment with name 'iris-comparison-cv' does not exist. Creating a new experiment.


🚀 Training and logging with CV: Logistic Regression


2025/09/17 12:07:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:07:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/878783076185509268/runs/59aea345d1854b2dac0ce6eec4348f8a
🧪 View experiment at: http://localhost:5000/#/experiments/878783076185509268
🚀 Training and logging with CV: Random Forest


2025/09/17 12:07:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:07:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/878783076185509268/runs/f0986ceb0a96471e89366270a53d15ca
🧪 View experiment at: http://localhost:5000/#/experiments/878783076185509268
🚀 Training and logging with CV: SVM


2025/09/17 12:07:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:07:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SVM at: http://localhost:5000/#/experiments/878783076185509268/runs/f89d4db6b153474f86bdc042e1bbbb28
🧪 View experiment at: http://localhost:5000/#/experiments/878783076185509268

🎯 All models trained with CV, logged, and tracked in MLflow.


In [9]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# =====================
# Setup MLflow
# =====================
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("iris-comparison-different")

# =====================
# Load dataset (with noise to make task harder)
# =====================
iris = load_iris()
X, y = iris.data, iris.target

# Add small Gaussian noise (forces differences)
rng = np.random.RandomState(42)
X_noisy = X + rng.normal(0, 0.3, X.shape)

# Smaller training set for more variability
X_train, X_test, y_train, y_test = train_test_split(
    X_noisy, y, test_size=0.35, random_state=7, stratify=y
)

# =====================
# Define models with different capacities
# =====================
models = {
    "Logistic Regression": LogisticRegression(max_iter=100, solver="liblinear", C=0.2),  # underfit
    "Random Forest": RandomForestClassifier(n_estimators=20, max_depth=2, random_state=42),  # limited depth
    "SVM": SVC(kernel="rbf", probability=True, C=1.5, gamma=0.5, random_state=42)  # more complex
}

# Directory for plots
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

# =====================
# Train and Log each model
# =====================
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        print(f"🚀 Training and logging: {model_name}")

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # ---- Metrics ----
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average="macro")
        rec = recall_score(y_test, y_pred, average="macro")
        f1 = f1_score(y_test, y_pred, average="macro")

        print(f"{model_name} - Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")

        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        mlflow.log_metric("recall", rec)
        mlflow.log_metric("f1_score", f1)

        # ---- Parameters ----
        if isinstance(model, LogisticRegression):
            mlflow.log_param("C", model.C)
            mlflow.log_param("solver", model.solver)
        elif isinstance(model, RandomForestClassifier):
            mlflow.log_param("n_estimators", model.n_estimators)
            mlflow.log_param("max_depth", model.max_depth)
        elif isinstance(model, SVC):
            mlflow.log_param("kernel", model.kernel)
            mlflow.log_param("C", model.C)
            mlflow.log_param("gamma", model.gamma)

        # ---- Confusion Matrix ----
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=iris.target_names,
                    yticklabels=iris.target_names)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix - {model_name}")

        plot_path = os.path.join(results_dir, f"{model_name}_confusion_matrix.png")
        plt.savefig(plot_path)
        plt.close()

        mlflow.log_artifact(plot_path, artifact_path="plots")

        # ---- Log Model ----
        mlflow.sklearn.log_model(model, artifact_path="model")

print("\n🎯 All models trained and logged in MLflow with DIFFERENT accuracies.")


2025/09/17 12:09:43 INFO mlflow.tracking.fluent: Experiment with name 'iris-comparison-different' does not exist. Creating a new experiment.


🚀 Training and logging: Logistic Regression
Logistic Regression - Accuracy: 0.774, Precision: 0.823, Recall: 0.777, F1: 0.755


2025/09/17 12:09:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:09:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/839992748732633827/runs/5c82a53f91e2468abbe74d1d56b3ffe7
🧪 View experiment at: http://localhost:5000/#/experiments/839992748732633827
🚀 Training and logging: Random Forest
Random Forest - Accuracy: 0.849, Precision: 0.847, Recall: 0.847, F1: 0.847


2025/09/17 12:09:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:09:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/839992748732633827/runs/602657dd26e84439a74f620f8ab83648
🧪 View experiment at: http://localhost:5000/#/experiments/839992748732633827
🚀 Training and logging: SVM
SVM - Accuracy: 0.906, Precision: 0.905, Recall: 0.905, F1: 0.905


2025/09/17 12:10:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:10:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SVM at: http://localhost:5000/#/experiments/839992748732633827/runs/563807c6e471486ea7305d9859b05a5c
🧪 View experiment at: http://localhost:5000/#/experiments/839992748732633827

🎯 All models trained and logged in MLflow with DIFFERENT accuracies.


In [10]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# =====================
# Setup MLflow
# =====================
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("iris-comparison-different")

# =====================
# Load dataset (with noise to make task harder)
# =====================
iris = load_iris()
X, y = iris.data, iris.target

# Add Gaussian noise so models behave differently
rng = np.random.RandomState(42)
X_noisy = X + rng.normal(0, 0.3, X.shape)

# Smaller training set = harder problem
X_train, X_test, y_train, y_test = train_test_split(
    X_noisy, y, test_size=0.35, random_state=7, stratify=y
)

# =====================
# Define models
# =====================
models = {
    "Logistic Regression": LogisticRegression(max_iter=100, solver="liblinear", C=0.2),  # underfit
    "Random Forest": RandomForestClassifier(n_estimators=20, max_depth=2, random_state=42),  # limited depth
    "SVM": SVC(kernel="rbf", probability=True, C=1.5, gamma=0.5, random_state=42)  # more complex
}

# Directory for results
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

# =====================
# Train & Log models
# =====================
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        print(f"\n🚀 Training and logging: {model_name}")

        # Train
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # ---- Metrics ----
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average="macro")
        rec = recall_score(y_test, y_pred, average="macro")
        f1 = f1_score(y_test, y_pred, average="macro")

        print(f"{model_name} - Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")

        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        mlflow.log_metric("recall", rec)
        mlflow.log_metric("f1_score", f1)

        # ---- Parameters ----
        if isinstance(model, LogisticRegression):
            mlflow.log_param("C", model.C)
            mlflow.log_param("solver", model.solver)
        elif isinstance(model, RandomForestClassifier):
            mlflow.log_param("n_estimators", model.n_estimators)
            mlflow.log_param("max_depth", model.max_depth)
        elif isinstance(model, SVC):
            mlflow.log_param("kernel", model.kernel)
            mlflow.log_param("C", model.C)
            mlflow.log_param("gamma", model.gamma)

        # ---- Confusion Matrix ----
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=iris.target_names,
                    yticklabels=iris.target_names)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix - {model_name}")

        plot_path = os.path.join(results_dir, f"{model_name}_confusion_matrix.png")
        plt.savefig(plot_path)
        plt.close()

        mlflow.log_artifact(plot_path, artifact_path="plots")

        # ---- Log Model ----
        mlflow.sklearn.log_model(model, artifact_path="model")

print("\n🎯 All 3 models trained and logged in MLflow with DIFFERENT accuracies.")



🚀 Training and logging: Logistic Regression
Logistic Regression - Accuracy: 0.774, Precision: 0.823, Recall: 0.777, F1: 0.755


2025/09/17 12:10:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:10:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/839992748732633827/runs/144d7176db5b4adeb57b6194b32deec9
🧪 View experiment at: http://localhost:5000/#/experiments/839992748732633827

🚀 Training and logging: Random Forest
Random Forest - Accuracy: 0.849, Precision: 0.847, Recall: 0.847, F1: 0.847


2025/09/17 12:10:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:10:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/839992748732633827/runs/e1becf6865724630a85cf23d152625f9
🧪 View experiment at: http://localhost:5000/#/experiments/839992748732633827

🚀 Training and logging: SVM
SVM - Accuracy: 0.906, Precision: 0.905, Recall: 0.905, F1: 0.905


2025/09/17 12:11:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:11:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SVM at: http://localhost:5000/#/experiments/839992748732633827/runs/8cf928fb0bc64eee9806954be8db4cb8
🧪 View experiment at: http://localhost:5000/#/experiments/839992748732633827

🎯 All 3 models trained and logged in MLflow with DIFFERENT accuracies.


In [11]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# =====================
# Setup MLflow
# =====================
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("iris-comparison-different")

# =====================
# Load dataset (with noise to make task harder)
# =====================
iris = load_iris()
X, y = iris.data, iris.target

# Add Gaussian noise so models behave differently
rng = np.random.RandomState(42)
X_noisy = X + rng.normal(0, 0.3, X.shape)

# Smaller training set = harder problem
X_train, X_test, y_train, y_test = train_test_split(
    X_noisy, y, test_size=0.35, random_state=7, stratify=y
)

# =====================
# Define models
# =====================
models = {
    "Logistic Regression": LogisticRegression(max_iter=100, solver="liblinear", C=0.2),
    "Random Forest": RandomForestClassifier(n_estimators=20, max_depth=2, random_state=42),
    "SVM": SVC(kernel="rbf", probability=True, C=1.5, gamma=0.5, random_state=42)
}

# Directory for results
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

# Store results for comparison
results = []

# =====================
# Train & Log models
# =====================
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name) as run:
        print(f"\n🚀 Training and logging: {model_name}")

        # Train
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # ---- Metrics ----
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average="macro")
        rec = recall_score(y_test, y_pred, average="macro")
        f1 = f1_score(y_test, y_pred, average="macro")

        print(f"{model_name} - Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")

        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        mlflow.log_metric("recall", rec)
        mlflow.log_metric("f1_score", f1)

        # ---- Parameters ----
        if isinstance(model, LogisticRegression):
            mlflow.log_param("C", model.C)
            mlflow.log_param("solver", model.solver)
        elif isinstance(model, RandomForestClassifier):
            mlflow.log_param("n_estimators", model.n_estimators)
            mlflow.log_param("max_depth", model.max_depth)
        elif isinstance(model, SVC):
            mlflow.log_param("kernel", model.kernel)
            mlflow.log_param("C", model.C)
            mlflow.log_param("gamma", model.gamma)

        # ---- Confusion Matrix ----
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=iris.target_names,
                    yticklabels=iris.target_names)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix - {model_name}")

        plot_path = os.path.join(results_dir, f"{model_name}_confusion_matrix.png")
        plt.savefig(plot_path)
        plt.close()

        mlflow.log_artifact(plot_path, artifact_path="plots")

        # ---- Log Model ----
        mlflow.sklearn.log_model(model, artifact_path="model")

        # Save results for comparison
        results.append({
            "model_name": model_name,
            "run_id": run.info.run_id,
            "accuracy": acc,
            "model": model
        })

print("\n🎯 All models trained and logged in MLflow.")

# =====================
# Select Best Model & Register
# =====================
best_model = max(results, key=lambda x: x["accuracy"])
print(f"\n🏆 Best Model: {best_model['model_name']} (Accuracy={best_model['accuracy']:.3f})")

# Register in MLflow Model Registry
model_uri = f"runs:/{best_model['run_id']}/model"
mlflow.register_model(model_uri, "IrisBestModel")

print("\n✅ Best model registered in MLflow Model Registry as 'IrisBestModel'")



🚀 Training and logging: Logistic Regression
Logistic Regression - Accuracy: 0.774, Precision: 0.823, Recall: 0.777, F1: 0.755


2025/09/17 12:14:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:14:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/839992748732633827/runs/a596c922705b48e5bfb222a1f011b260
🧪 View experiment at: http://localhost:5000/#/experiments/839992748732633827

🚀 Training and logging: Random Forest
Random Forest - Accuracy: 0.849, Precision: 0.847, Recall: 0.847, F1: 0.847


2025/09/17 12:15:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:15:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/839992748732633827/runs/98859bcade9b4478a3d16783e277edc1
🧪 View experiment at: http://localhost:5000/#/experiments/839992748732633827

🚀 Training and logging: SVM
SVM - Accuracy: 0.906, Precision: 0.905, Recall: 0.905, F1: 0.905


2025/09/17 12:15:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 12:15:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'IrisBestModel' already exists. Creating a new version of this model...
2025/09/17 12:15:12 WARNING mlflow.tracking._model_registry.fluent: Run with id a3f44d0673124eb2af83b3a866db24dd has no artifacts at artifact path 'model', registering model based on models:/m-f589262812a54ce58d16684046a4ec50 instead


🏃 View run SVM at: http://localhost:5000/#/experiments/839992748732633827/runs/a3f44d0673124eb2af83b3a866db24dd
🧪 View experiment at: http://localhost:5000/#/experiments/839992748732633827

🎯 All models trained and logged in MLflow.

🏆 Best Model: SVM (Accuracy=0.906)


2025/09/17 12:15:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IrisBestModel, version 2
Created version '2' of model 'IrisBestModel'.



✅ Best model registered in MLflow Model Registry as 'IrisBestModel'
